In [ ]:
import numpy as np
import pika
import json
import time

class WorldComponent:
    def __init__(self):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        self.channel = self.connection.channel()
        
        queue_args = {
            "x-max-length": 1  # Set maximum number of messages in the queue to 1
        }
        
        self.channel.queue_declare(queue='world_update_queue', arguments=queue_args)  # Make queue persistent
        self.channel.queue_declare(queue='terrain_init', arguments=queue_args, durable=True)  # Make queue persistent

        self.tick = 0
        self.terrain_image_path = "sample_world_terrain.png"
        self.init_terrain_image()

    def init_terrain_image(self):
        with open(self.terrain_image_path, 'rb') as image_file:
            image_binary = image_file.read()
            self.channel.basic_publish(exchange='',
                                       routing_key='terrain_init',
                                       body=image_binary,
                                       properties=pika.BasicProperties(
                                           delivery_mode=2,  # Mach die Nachricht dauerhaft
                                       ))

    def update(self):
        self.tick += 1
        new_world_state = {
            "tick": self.tick,
            "entities": ["entity_1", "entity_2"]
        }
        self.channel.basic_publish(exchange='',
                                   routing_key='world_update_queue',
                                   body=json.dumps(new_world_state),
                                   properties=pika.BasicProperties(delivery_mode=2))  # Make message persistent

    def run(self):
        while True:
            self.update()
            self.init_terrain_image()
            time.sleep(1)

# Instanz der WorldComponent-Klasse
world_component = WorldComponent()

# Start der dauerhaften Updateschleife
world_component.run()